$$\textrm{Joaquin Peñuela Parra, Cristian Fernando Rodriguez Cruz}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [ ]:
!rm -rf Uniandes_Framework
!git clone https://github.com/Phenomenology-group-uniandes/Uniandes_Framework.git

In [ ]:
import os, sys
sys.path.append(f'{os.getcwd()}/Uniandes_Framework')

from delphes_reader import DelphesLoader 
from delphes_reader import clasificator 
from delphes_reader import root_analysis 
from delphes_reader import Quiet 

from ROOT import TChain
from ROOT import TLorentzVector

import pandas as pd
from multiprocessing import Pool

In [ ]:
def event_selection(signal):
    
    csv_folder_path = f'/disco4/pheno_csv_files/Leptoquarks_Searches/{signal}/'
    !mkdir -p {csv_folder_path}    
    
    with Quiet():
    
        CUTS={"l_jet":     {"pt_min_cut":30., "eta_min_cut":-5.,  "eta_max_cut":+5. },
              "b_jet":     {"pt_min_cut":30., "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "tau_jet":   {"pt_min_cut":50., "eta_min_cut":-2.3, "eta_max_cut":+2.3},
              "other_jet": {"pt_min_cut":0.,  "eta_min_cut":-5.,  "eta_max_cut":+5.},
              "electron":  {"pt_min_cut":35.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "muon":      {"pt_min_cut":30.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4}}

        ST = []
        
        Delphes = DelphesLoader(signal)
        Paths = Delphes.Forest
        xs = Delphes.xs   
        
        #We want a Cutflows file for each cathegory:        
        Cutflows = {'hadronic_non-resonant': {'XS': xs},
                    'hadronic_sLQ': {'XS': xs},
                    'hadronic_dLQ': {'XS': xs},
                    'semileptonic_non-resonant': {'XS': xs},
                    'semileptonic_sLQ': {'XS': xs},
                    'semileptonic_dLQ': {'XS': xs}}
        
        kinematic_variables = {}
        for key in Cutflows.keys(): kinematic_variables[key] = []
        
        for path in Paths:

            tree = TChain("Delphes;1") 
            tree.Add(path) 
                        
            for event in tree:
                
                for key in Cutflows.keys(): Cutflows[key]['All'] = Cutflows[key].get('All',0) + 1
                    
                leptons = clasificator.get_good_leptons(event, kin_cuts= CUTS) 
                
                if(len(leptons) > 1): continue
                for key in Cutflows.keys(): Cutflows[key]['Maximum 1 light lepton'] = Cutflows[key].get('Maximum 1 light lepton',0) + 1
                           
                jets = clasificator.get_good_jets(event, kin_cuts= CUTS)
                
                #Adding tau_jets like leptons:
                leptons = leptons + jets['tau_jet'] 
                
                if(len(leptons) != 2): continue 
                for key in Cutflows.keys(): Cutflows[key]['Exactly 2 leptons'] = Cutflows[key].get('Exactly 2 leptons',0) + 1

                #At this point, we have two hadronic taus or one hadronic tau and one lepton. Now, we can clasificate:
                
                #clasification:
                if(len(jets['tau_jet']) == 1): 
                    label1 = 'semileptonic'
                else:
                    label1 = 'hadronic'  
                
                    
                cut = f'{label1} selection'    
                for key in Cutflows.keys(): 
                    if label1 in key: Cutflows[key][cut] = Cutflows[key].get(cut, 0) + 1
                
                if(len(jets['b_jet']) == 0):
                    label2 = 'non-resonant'
                elif(len(jets['b_jet']) == 1):
                    label2 = 'sLQ'
                elif(len(jets['b_jet']) == 2):
                    label2 = 'dLQ'
                else: continue
                
                label = f'{label1}_{label2}'  
                
                cut = f'{label2} selection'    
                Cutflows[label][cut] = Cutflows[label].get(cut, 0) + 1                
                
                if(leptons[0].DeltaR(leptons[1]) < 0.3): continue #DeltaR > 0.3
                cut = 'DeltaR > 0.3'
                Cutflows[label][cut] = Cutflows[label].get(cut,0) + 1
                
                MET = clasificator.get_met(event)
                MET.SetName('MET')
                particles = leptons + jets['b_jet'] + [MET]
                
                ST = 0
                for particle in particles: ST += particle.pt()
                
                HT = 0
                for particle in jets['l_jet']: HT += particle.pt()
                
                TLV = TLorentzVector(0,0,0,0)
                for particle in particles: TLV += particle.GetTLV()
                MT = TLV.Mt()
                
                Mass_invariant = (leptons[0].GetTLV() + leptons[1].GetTLV()).M()
                
                row = root_analysis.get_kinematics_row(particles)
                
                keylist = list(row.keys())
                for key in keylist: 
                    if ("Mass" in key): row.pop(key,None)
                
                row["sT(GeV)"] = ST
                row["hT(GeV)"] = HT
                row["mT(GeV)"] = MT
                row["light_jets_multiplicity"] = len(jets['l_jet'])
                name = leptons[0].Name + leptons[1].Name
                row[f"Mass_{{{name}}}(GeV)"] = Mass_invariant
                row[f"Q_{{{leptons[0].Name}}}Q_{{{leptons[1].Name}}}"] = leptons[0].Charge*leptons[1].Charge
                
                
                kinematic_variables[label].append(row)
                                
        for key in Cutflows.keys(): root_analysis.generate_csv(kinematic_variables[key], os.path.join(csv_folder_path, f'{signal}_{key}.csv'))
                
    return {signal: Cutflows}

In [ ]:
Masses = ['1250', '1500', '1750', '2000', '2250', '2500']
LQ_signals = ['LQ_LQ', 'Tau_LQ', 'Tau_Tau']

signals = []

for signal in LQ_signals:
    for M in Masses:
        signals.append(f'{signal}_{M}')
        
signals = signals + ['ttbar', 'stop','z_jets', 'w_jets', 'ww', 'wz', 'zz']

In [ ]:
with Pool(6) as p:
    mapping = list(p.map(event_selection, signals))

In [ ]:
labels = ['hadronic_non-resonant','hadronic_sLQ','hadronic_dLQ','semileptonic_non-resonant','semileptonic_sLQ','semileptonic_dLQ']

Dict_Cutflows = {}

for label in labels: Dict_Cutflows[label] = {}

for Cutflows_Signal_Directory in mapping:
    
    signal = list(Cutflows_Signal_Directory.keys())[0]
    
    for label in labels: Dict_Cutflows[label][signal] = Cutflows_Signal_Directory[signal][label]

In [ ]:
csv_folder_path = f'/disco4/pheno_csv_files/Leptoquarks_Searches/'

Dict_DataFrames_Cutflows = {}

for key in Dict_Cutflows: 
    Dict_DataFrames_Cutflows[key] = pd.DataFrame.from_dict(Dict_Cutflows[key])
    carpeta = os.path.join(csv_folder_path, 'cutflows')
    try: os.mkdir(carpeta)
    except: pass
    Dict_DataFrames_Cutflows[key].to_csv(os.path.join(carpeta, f'{key}.csv'))